# Podstawy programowania w języku Python - Projekt 
## Gra Sudoku
### Natalia Szwagierczak 73207

### Spis tresci
1. [Generowanie plansz](#Generowanie-plansz)<br>
2. [Nakladanie maski](#Nakladanie-maski)<br>
3. [Autonomiczne rozwiazywanie Sudoku](#Autonomiczne-rozwiazywanie-Sudoku)<br>
* [Rozwiazywanie Sudoku w aplikacji](#Rozwiazywanie-Sudoku-w-aplikacji)<br>
* [Sprawdzanie poprawnosci rozwiazania](#Sprawdzanie-poprawnosci-rozwiazania)<br>
* [Tworzenie GUI](#Tworzenie-GUI)<br>
 - [Okienka danych wejściowych](#Okienka-danych-wejściowych)<br>
 - [Obramowania](#Obramowania)<br><br>
* [Aktualizowanie: sprawdzanie poprawności danych wejściowych](#Aktualizowanie:-sprawdzanie-poprawności-danych-wejściowych)<br>
* [Graficzny interfejs uzytkownika](#Graficzny-interfejs-uzytkownika)<br>
* [Inicjalizacja gry GUI](#Inicjalizacja-gry-GUI)<br>
* [Testowanie](#Testowanie)<br>
 - [Testowanie non-GUI](#Testowanie-non-GUI)<br>
 - [Testowanie Sudoku non-GUI o roznych poziomach](#Testowanie-Sudoku-non-GUI-o-roznych-poziomach)<br>
 - [Testowanie GUI](#Testowanie-GUI)<br><br>
* [Gra](#Gra)<br>
<br>Gra otwiera sie w osobnym okienku

In [1]:
try:
    import tkinter as tk
except ImportError:
    import Tkinter as tk # rozne wersje
import numpy as np
import sys
import time
import tkinter.messagebox


### Zmienne globalne

In [2]:
# Wymiary okienka graficznego interfejsu uzytkownika
HEIGHT=500
WIDTH =500
MARGIN= 50
SIDE=50
entries=[]
cl=None

### Generowanie plansz

In [3]:
def generate_sudoku():
    while True:
        n = 9
        m = np.zeros((n, n), np.int)
        rg = np.arange(1, n + 1)
        m[0, :] = np.random.choice(rg, n, replace=False)
        try:
            for r in range(1, n):
                for c in range(n):
                    col_rest = np.setdiff1d(rg, m[:r, c])
                    row_rest = np.setdiff1d(rg, m[r, :c])
                    avb1 = np.intersect1d(col_rest, row_rest)
                    sub_r, sub_c = r//3, c//3
                    avb2 = np.setdiff1d(np.arange(0, n+1), m[sub_r*3:(sub_r+1)*3, sub_c*3:(sub_c+1)*3].ravel())
                    avb = np.intersect1d(avb1, avb2)
                    m[r, c] = np.random.choice(avb, size=1)
            break
        except ValueError:
            pass
    print("Odpowiedz:\n", m)
    return m

### Nakladanie maski

In [4]:
def mask(m,mask_rate=0.5):
    mm = m.copy()
    mm[np.random.choice([True, False], size=m.shape, p=[mask_rate, 1 - mask_rate])] = 0 #maskowanie
    print("\nOdpowiedz z maska:\n", mm)
    #np.savetxt("./board.csv", mm, "%d", delimiter=",")
    return mm

### Autonomiczne rozwiazywanie Sudoku

In [5]:
def solve(m):
    if isinstance(m, list):
        m = np.array(m)
    rg = np.arange(m.shape[0]+1)
    while True:
        mt = m.copy()
        while True:
            d = []
            d_len = []
            for i in range(m.shape[0]):
                for j in range(m.shape[1]):
                    if mt[i][j] == 0:
                        possibles = np.setdiff1d(rg, np.union1d(np.union1d(mt[i, :], mt[:, j]), mt[3*(i//3):3*(i//3+1), 3*(j//3):3*(j//3+1)]))
                        d.append([i, j, possibles])
                        d_len.append(len(possibles))
            if len(d) == 0:
                break
            idx = np.argmin(d_len)
            i, j, p = d[idx]
            if len(p) > 0:
                num = np.random.choice(p)
            else:
                break
            mt[i, j] = num
            if len(d) == 0:
                break
        if np.all(mt != 0):
            break

    print("\nRozwiazanie z solvera:\n", mt)
    return mt

### Rozwiazywanie Sudoku w aplikacji

In [6]:
def sol(top,board,canvas, solution):
    global cl
    global entries
    bl=False
    p,q=SIDE+1,SIDE+1
    for i in range(9):
        for j in range(9):
            if(board[i][j]!=solution[i][j] and board[i][j]!=0):
                bl=True
                entries[i*9+j].config({"background": "Red"})
            elif board[i][j]==solution[i][j]:
                try:
                    entries[i*9+j].config({"background": "Green"})
                except:
                    pass
            p+=SIDE-9
        q+=SIDE-9
        p=SIDE+1
        
    p,q=SIDE+1,SIDE+1
    if bl==False:    
        for i in range(9):
            for j in range(9):
                if(board[i][j]==0):
                    entries[i*9+j].config({"background": "LightBlue"})
                    entries[i*9+j].delete(0,tk.END)
                    entries[i*9+j].insert(0,solution[i][j])
                    #canvas.create_text(p+SIDE/2,q+SIDE/2,fill="darkblue", font=('Arial',int(SIDE/3)), text=solution[i][j])
                    
                p+=SIDE-9
            q+=SIDE-9
            p=SIDE+1
    else:
        MsgBox = tk.messagebox.askquestion ('Info','Wprowadzono niepoprawne wartosci. Czy chcesz je poprawic?',icon = 'warning')
        if MsgBox == 'no':
            for i in range(9):
                for j in range(9):
                    try:
                        entries[i*9+j].delete(0,tk.END)
                        entries[i*9+j].insert(0,solution[i][j])
                    except:
                        pass

### Sprawdzanie poprawnosci rozwiazania

In [7]:
def check_solution(m):
    if isinstance(m, list):
        m = np.array(m)
    set_rg = set(np.arange(1, m.shape[0] + 1))
    no_good = False
    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            r1 = set(m[3 * (i // 3):3 * (i // 3 + 1), 3 * (j // 3):3 * (j // 3 + 1)].ravel()) == set_rg
            r2 = set(m[i, :]) == set_rg
            r3 = set(m[:, j]) == set_rg
            if not (r1 and r2 and r3):
                no_good = True
                break
        if no_good:
            break
    if no_good:
        print("Rozwiazanie niepoprawne")
        return "Tym razem nie udalo sie rozwiazac Sudoku.\nSprobuj ponownie"
    else:
        print("Rozwiazanie poprawne")
        return "Gratulacje! Sudoku rozwiazane poprawnie"

### Tworzenie GUI

#### Okienka danych wejściowych

In [8]:
def createEntry(top,board, canvas):
    global cl
    global entries
    E=None
    p,q=SIDE+1, SIDE+1
    for i in range(9):
        for j in range(9):
            if(board[i][j]==0):
                E = tk.Entry(top, width=3,font=('Arial',int(SIDE/3)))
                E.grid(row=i, column=j)
                E.place(x=p, y=q, height=SIDE-9-3, width=SIDE-9-3)
                entries.append(E)
            else:
                entries.append(board[i][j])
                canvas.create_text(p+SIDE/2,q+SIDE/2,fill="darkblue", font=('Arial',int(SIDE/3)), text=board[i][j])
            p+=SIDE-9
        q+=SIDE-9
        p=SIDE+1

    cl=type(E)

#### Obramowania

In [9]:
def createRow(canvas):
    i,j=SIDE, SIDE
    p=SIDE
    q=SIDE*9-SIDE*2/3+4
    for m in range(10):
        if(m%3==0):
            canvas.create_line(i,j,p,q,width=4,fill="SteelBlue1")
        else:
            canvas.create_line(i,j,p,q,width=0.5,fill="LightSteelBlue3")
        i+=SIDE-9
        p+=SIDE-9
    
def createCol(canvas):
    i,j=SIDE, SIDE
    p,q=SIDE*9-SIDE*2/3+4,SIDE
    for m in range(10):
        if(m%3==0):
            canvas.create_line(i,j,p,q,width=4,fill="SteelBlue1")
        else:
            canvas.create_line(i,j,p,q,width=0.5,fill="LightSteelBlue3")
        j+=SIDE-9
        q+=SIDE-9

In [10]:
def multiple_choice(prompt, options):
    root = tk.Tk()
    if prompt:
        tk.Label(root, text=prompt).pack()
    v = tk.IntVar()
    for i, option in enumerate(options):
        tk.Radiobutton(root, text=option, variable=v, value=i).pack(anchor="w")
    tk.Button(text="OK", command=root.destroy).pack()
    root.attributes("-topmost", True)
    root.mainloop()
    
    #if v.get() == 0: return None
    return options[v.get()]

### Aktualizowanie: sprawdzanie poprawności danych wejściowych

In [11]:
def update(canvas,top,board):
    global entries
    if(len(entries)>81):
        entries=entries[:81]
    for i in range(9):
        for j in range(9):
            E = entries[i*9+j]
            if type(E)==cl:
                if E.get(): # jesli jest wprowadzona wartosc do komorki
                    if E.get().isdigit():
                        if (int(E.get())>=0 and int(E.get())<=9):
                            board[i][j] = int(E.get())
                            if entries[i*9+j]["background"]=='Red':
                                entries[i*9+j].config({"background": "White"})
                        else:
                            board[i][j] =0 
                            tk.messagebox.showinfo("Error", "Wiersz: %d, kolumna: %d\nWprowadzone wartosci powinny byc z przedzialu <1;9>" %(i+1,j+1))
                            
                            entries[i*9+j].config({"background": "Red"})
                    else:
                        board[i][j] =0
                        if E.get()!="":
                            tk.messagebox.showinfo("Error", "Wiersz: %d, kolumna: %d\nWprowadzone wartosci powinny byc cyframi" %(i+1,j+1))
                            entries[i*9+j].config({"background": "Red"})
    return board

### Graficzny interfejs uzytkownika

In [12]:
def createGUI(board,test=None):
    sys.stdout.flush() # czyszczenie buffora
    solved = solve(board)
    entries=[]
    top = tk.Tk()
    top.title("Natalia Szwagierczak - projekt zaliczeniowy - Sudoku")
    canvas = tk.Canvas(top, height=HEIGHT, width =WIDTH)
    
    createEntry(top,board,canvas)
    createCol(canvas)
    createRow(canvas)
    button_get = tk.Button(top, text="Sprawdz", justify='right', command = lambda: [update(canvas,top,board),print(board),tk.messagebox.showinfo("Wynik", check_solution(board))])
    button_get.place(x=WIDTH/5, y=HEIGHT-25, height=25, width=60)
    button_new = tk.Button(top, text="Nowa gra", justify='right', command = lambda: [top.destroy(), play()])
    button_new.place(x=WIDTH/5*2, y=HEIGHT-25, height=25, width=60)
    button_solve = tk.Button(top, text="Rozwiaz", justify='right', command = lambda: [sol(top,update(canvas,top,board),canvas, solved)])
    button_solve.place(x=WIDTH*3/5, y=HEIGHT-25, height=25, width=60)
    
    canvas.pack(side = 'top')
    if test is not None:
        top.after(int(test/4*3),sol(top,update(canvas,top,board),canvas, solved))
        canvas.create_text(WIDTH/2,int(SIDE/3),fill="darkblue", font=('Arial',int(SIDE/3)), text="Tryb testow")
        top.after(test, top.destroy)
    top.attributes("-topmost", True)
    top.mainloop()
    

### Inicjalizacja gry GUI

In [13]:
def play():
    sys.stdout.flush() # czyszczenie buffora
    print("Uwaga! Gra uruchamia sie w nowym oknie")
    d={"bardzo latwy": 0.05, "latwy": 0.1, "sredni":0.4, "trudny":0.5, "bardzo trudny":0.7}
    res=d[multiple_choice("Wybierz poziom:", list(d.keys()))]
    global entries
    sys.stdout.flush() # czyszczenie buffora
    entries=None
    board = generate_sudoku()
    board = mask(board, mask_rate=res)
    entries=[]
    createGUI(board)

## Testowanie

### Testowanie non-GUI

In [14]:
test_board=generate_sudoku()

Odpowiedz:
 [[8 2 9 1 5 7 3 6 4]
 [4 6 3 8 9 2 7 5 1]
 [7 5 1 3 4 6 9 8 2]
 [6 8 5 2 7 4 1 9 3]
 [2 1 7 5 3 9 8 4 6]
 [9 3 4 6 8 1 5 2 7]
 [3 4 8 7 6 5 2 1 9]
 [1 7 6 9 2 8 4 3 5]
 [5 9 2 4 1 3 6 7 8]]


In [15]:
test_masked=mask(test_board,mask_rate=0.6)


Odpowiedz z maska:
 [[8 0 0 0 5 0 3 0 0]
 [0 0 0 0 0 0 7 0 1]
 [0 0 1 0 0 0 0 0 2]
 [6 0 5 0 0 0 1 9 3]
 [2 1 0 0 3 0 8 4 0]
 [9 3 0 0 0 0 0 2 0]
 [3 4 8 0 0 0 0 0 0]
 [0 0 6 0 2 0 4 3 0]
 [0 9 0 0 0 3 0 0 0]]


In [16]:
solve(test_masked)


Rozwiazanie z solvera:
 [[8 2 9 7 5 1 3 6 4]
 [5 6 3 4 9 2 7 8 1]
 [4 7 1 3 6 8 9 5 2]
 [6 8 5 2 7 4 1 9 3]
 [2 1 7 9 3 6 8 4 5]
 [9 3 4 1 8 5 6 2 7]
 [3 4 8 5 1 9 2 7 6]
 [1 5 6 8 2 7 4 3 9]
 [7 9 2 6 4 3 5 1 8]]


array([[8, 2, 9, 7, 5, 1, 3, 6, 4],
       [5, 6, 3, 4, 9, 2, 7, 8, 1],
       [4, 7, 1, 3, 6, 8, 9, 5, 2],
       [6, 8, 5, 2, 7, 4, 1, 9, 3],
       [2, 1, 7, 9, 3, 6, 8, 4, 5],
       [9, 3, 4, 1, 8, 5, 6, 2, 7],
       [3, 4, 8, 5, 1, 9, 2, 7, 6],
       [1, 5, 6, 8, 2, 7, 4, 3, 9],
       [7, 9, 2, 6, 4, 3, 5, 1, 8]])

In [17]:
check_solution(solve(test_masked))


Rozwiazanie z solvera:
 [[8 7 9 1 5 2 3 6 4]
 [5 2 3 4 6 9 7 8 1]
 [4 6 1 3 8 7 9 5 2]
 [6 8 5 2 7 4 1 9 3]
 [2 1 7 9 3 6 8 4 5]
 [9 3 4 5 1 8 6 2 7]
 [3 4 8 7 9 5 2 1 6]
 [7 5 6 8 2 1 4 3 9]
 [1 9 2 6 4 3 5 7 8]]
Rozwiazanie poprawne


'Gratulacje! Sudoku rozwiazane poprawnie'

### Testowanie Sudoku non-GUI o roznych poziomach

In [18]:
sys.stdout.flush() # czyszczenie buffora
for mask_rate in [0.6,0.4,0.2]:
    for i in range(2):
        print("--- iteration:", i, "mask rate:", mask_rate, "---\n")
        test_board=generate_sudoku()
        test_masked=mask(test_board,mask_rate=0.6)
        check_solution(solve(test_masked))

--- iteration: 0 mask rate: 0.6 ---

Odpowiedz:
 [[7 9 1 2 5 4 3 8 6]
 [3 4 8 6 1 9 7 2 5]
 [5 2 6 8 3 7 9 1 4]
 [9 8 4 5 2 6 1 7 3]
 [1 6 3 7 9 8 4 5 2]
 [2 7 5 1 4 3 6 9 8]
 [4 5 2 9 6 1 8 3 7]
 [8 3 9 4 7 5 2 6 1]
 [6 1 7 3 8 2 5 4 9]]

Odpowiedz z maska:
 [[0 0 0 0 0 0 0 8 0]
 [3 0 0 6 0 0 7 0 5]
 [0 2 6 8 3 7 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 6 3 7 9 8 0 0 0]
 [0 7 0 1 0 3 0 0 8]
 [4 0 0 9 0 1 0 3 7]
 [8 0 0 0 0 5 0 6 1]
 [0 1 0 3 0 2 0 4 9]]

Rozwiazanie z solvera:
 [[7 4 1 2 5 9 3 8 6]
 [3 9 8 6 1 4 7 2 5]
 [5 2 6 8 3 7 9 1 4]
 [9 8 4 5 2 6 1 7 3]
 [1 6 3 7 9 8 4 5 2]
 [2 7 5 1 4 3 6 9 8]
 [4 5 2 9 6 1 8 3 7]
 [8 3 9 4 7 5 2 6 1]
 [6 1 7 3 8 2 5 4 9]]
Rozwiazanie poprawne
--- iteration: 1 mask rate: 0.6 ---

Odpowiedz:
 [[3 6 8 5 7 2 9 1 4]
 [7 1 9 3 8 4 2 6 5]
 [4 2 5 6 1 9 8 3 7]
 [2 5 4 9 6 1 7 8 3]
 [6 8 1 7 4 3 5 9 2]
 [9 7 3 8 2 5 6 4 1]
 [5 9 7 4 3 6 1 2 8]
 [8 3 2 1 9 7 4 5 6]
 [1 4 6 2 5 8 3 7 9]]

Odpowiedz z maska:
 [[0 0 0 0 0 2 0 0 0]
 [0 1 0 3 8 0 2 6 5]
 [0 0 0 0 1 

### Testowanie GUI

In [19]:
sys.stdout.flush() # czyszczenie buffora
for mask_rate in [0.6,0.4,0.2]:
    for i in range(2):
        print("----- iteration:",i,", mask rate:",mask_rate,"-----")
        entries=None
        board = generate_sudoku()
        test_board = mask(board, mask_rate=0.5)
        entries=[]
        createGUI(test_board,test=500)


----- iteration: 0 , mask rate: 0.6 -----
Odpowiedz:
 [[7 4 9 8 3 5 2 1 6]
 [5 6 1 9 7 2 4 3 8]
 [8 2 3 4 1 6 9 7 5]
 [4 5 2 6 8 1 3 9 7]
 [6 9 7 5 4 3 1 8 2]
 [3 1 8 2 9 7 6 5 4]
 [1 7 5 3 2 4 8 6 9]
 [9 3 4 7 6 8 5 2 1]
 [2 8 6 1 5 9 7 4 3]]

Odpowiedz z maska:
 [[7 0 0 8 0 5 0 1 6]
 [0 0 0 9 7 2 0 3 0]
 [0 0 0 4 0 6 0 7 0]
 [4 0 2 6 0 0 0 0 0]
 [6 9 7 0 0 3 1 0 2]
 [3 1 8 0 9 0 6 5 0]
 [1 7 0 3 2 0 8 6 0]
 [9 0 0 0 6 0 5 2 0]
 [2 8 0 1 0 9 7 0 3]]

Rozwiazanie z solvera:
 [[7 2 4 8 3 5 9 1 6]
 [8 6 1 9 7 2 4 3 5]
 [5 3 9 4 1 6 2 7 8]
 [4 5 2 6 8 1 3 9 7]
 [6 9 7 5 4 3 1 8 2]
 [3 1 8 2 9 7 6 5 4]
 [1 7 5 3 2 4 8 6 9]
 [9 4 3 7 6 8 5 2 1]
 [2 8 6 1 5 9 7 4 3]]
----- iteration: 1 , mask rate: 0.6 -----
Odpowiedz:
 [[2 6 7 5 9 8 3 4 1]
 [8 9 3 4 2 1 7 5 6]
 [1 5 4 3 6 7 8 2 9]
 [4 1 9 2 7 6 5 3 8]
 [6 8 5 1 4 3 9 7 2]
 [3 7 2 8 5 9 6 1 4]
 [7 4 8 6 1 5 2 9 3]
 [9 3 1 7 8 2 4 6 5]
 [5 2 6 9 3 4 1 8 7]]

Odpowiedz z maska:
 [[0 0 7 0 0 8 3 4 1]
 [8 0 0 0 0 1 0 0 0]
 [1 0 4 0 6 0 8 0 9]
 [

## Gra

In [ ]:
if __name__ == "__main__":
    sys.stdout.flush() # czyszczenie buffora
    play()

#### Graficzny interfejs uzytkownika uruchamia sie w nowym oknie